# Machine Learning with Flowers

In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau 

In [18]:
data = []

for dir, x, files in os.walk("flower_photos"): 
      label = dir.split("/")[-1]
  

      for file in files:
        path = os.path.join(dir, file)


        data.append([path,label])

#print(data)

df = pd.DataFrame(data, columns = ["file_path","label"])

print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [19]:
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [27]:
train_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical", 
    shuffle = "True", 
    subset = "training"
)

valid_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical", 
    shuffle = "True", 
    subset = "validation"
)



Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.
dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])


In [21]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

In [22]:
early_stopping = EarlyStopping(
    patience = 10,
    min_delta = 0,
    monitor = "val_loss",
    verbose = 0, 
    restore_best_weights = True,
    baseline = None
)

In [24]:
model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs = 5,
    validation_steps = None, 
    shuffle = False,
    callbacks = early_stopping, 
    verbose = 1
)

Epoch 1/5


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 309s 3s/step - acc: 0.3368 - auc_1: 0.6431 - loss: 1.9667 - precision_1: 0.3825 - recall_1: 0.2599 - val_acc: 0.2193 - val_auc_1: 0.6155 - val_loss: 1.9384 - val_precision_1: 0.2127 - val_recall_1: 0.1322
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - acc: 0.7342 - auc_1: 0.9262 - loss: 0.7689 - precision_1: 0.8052 - recall_1: 0.6599 - val_acc: 0.1240 - val_auc_1: 0.5135 - val_loss: 3.0619 - val_precision_1: 0.1294 - val_recall_1: 0.1049
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 284s 3s/step - acc: 0.8241 - auc_1: 0.9684 - loss: 0.5005 - precision_1: 0.8817 - recall_1: 0.7737 - val_acc: 0.1240 - val_auc_1: 0.4710 - val_loss: 3.8293 - val_precision_1: 0.1303 - val_recall_1: 0.1172
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 286s 3s/step - acc: 0.8833 - auc_1: 0.9840 - loss: 0.3478 - precision_1: 0.9147 - recall_1: 0.8565 - val_acc: 0.1240 - val_auc_1: 0.4441 - val_loss: 4.3726 - val_precision_1: 0.1278 - val_recall_1: 0.1172
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 274s 

In [25]:
model.save("flowers.keras") 

In [32]:
import cv2
from tensorflow.keras.saving import load_model 
import numpy as np 


model = load_model("flowers.keras") 

input_image = cv2.imread("flower_photos/daisy/5673551_01d1ea993e_n.jpg") 

input_image_resize = cv2.resize(input_image,  (224,224,))

input_image_scaled = input_image_resize/225

image_reshaped = np.reshape(input_image_scaled, [1, 224, 224, 3]) 



print(train_gen.class_indices.items())

input_prediction = model.predict(image_reshaped) 

print("your flower is a:", + np.argmax(input_prediction))

dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
your flower is a: 1
